In [ ]:
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('~')
from analysts_tools.growth import *
    #Procurement tools
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
from analysts_tools.redash_methods import *
from procurement_lib import redash, dw, send_message, send_slack_notification, GoogleSheet, logging

import random
import time
import datetime

: 

In [2]:
def info_sells(mes1):
    query = """
    select
        submit_date.full_date as fecha,
        site.identifier_value as region_code,
        sup.source_id::int as product_id,
        site.identifier_value || ' || ' || sup.description as product_name,
        dtd.dtd_cost_local as dtd_cost,
        b.min_price as benchmark,
        sum(s.product_price * s.product_quantity_x_step_unit) / nullif(sum(s.product_quantity_x_step_unit),0) as gross_price,
        sum(s.product_price_discount * s.product_quantity_x_step_unit) / nullif(sum(s.product_quantity_x_step_unit),0) as net_price,
        sum(s.product_quantity_x_step_unit) as quantity,
        sum(s.gmv_pxq_local) as gmv
    from dpr_sales.fact_sales s
        inner join dpr_shared.dim_site site
            on s.dim_site = site.site_id
        inner join dpr_shared.dim_date submit_date
            on s.dim_submitted_date = submit_date.date_id
        inner join dpr_shared.dim_stock_unit su
            on s.dim_product = su.product_id
        inner join dpr_shared.dim_stock_unit sup
            on nvl(nullif(su.source_parent_id,0),su.source_id) = sup.source_id
        inner join dpr_shared.dim_category cat
            on su.category_id = cat.category_id
        left join dpr_cross_business.int_dtd_cost dtd
            on submit_date.full_date-1 = to_date(dtd.dim_date_dtd::text,'YYYYMMDD')
            and sup.stock_unit_id = dtd.dim_stock_unit
        left join dpr_product_pricing.obt_benchmark_product_prices b
            on sup.stock_unit_id = b.stock_unit_id
            and submit_date.full_date = b.benchmark_date
    where submit_date.full_date >= DATE_TRUNC('MONTH', date(CURRENT_DATE) - interval '{mes1} MONTH')
        and submit_date.full_date < DATE_TRUNC('MONTH', date(CURRENT_DATE) - interval '{mes2} MONTH')
        and s.gmv_enabled = 'true'
        and cat.super_category = 'Fruver'
        and site.identifier_value in ('SPO','BHZ','CWB','VCP')
        and dtd.dtd_cost_local > 0
    group by 1,2,3,4,5,6
    """.format(mes1 = mes1, mes2= mes1-1)

    df_sells = run_read_dwd_query(query)
    
    return df_sells

In [3]:
def info_cost(mes1):
    query = """
with
execution_ids as(
    select max(execution_id)
    from lnd_ops.expected_dtd_costs
    where date >=  DATE_TRUNC('MONTH', date(CURRENT_DATE) - interval '{mes1} MONTH')
    AND date <= DATE_TRUNC('MONTH', date(CURRENT_DATE) - interval '{mes2} MONTH')
    group by date
)

select
    c.date+1 as fecha,
    c.region_code,
    c.product_id::int as product_id,
    c.cost_tomorrow as expected_dtd_cost
from lnd_ops.expected_dtd_costs c
    left join postgres_main_co."purchase_orders.products" p
        on c.product_id = p.frida_id
where c.execution_id in (select * from execution_ids)
    and c.region_code in ('SPO','BHZ','CWB','VCP')
    and p.product_category_id = 1
    """.format(mes1 = mes1, mes2= mes1-1)

    df_cost = read_connection_data_warehouse.runQuery(query)
    
    return df_cost

In [13]:
# Create an empty list to store DataFrames
dfs = []

for mes in range(0, 13):
    df1 = info_sells(mes)
    df2 = info_cost(mes)
    df3 = pd.merge(df1, df2[['product_id','fecha','expected_dtd_cost']],  how='left', left_on=['product_id','fecha'], right_on = ['product_id','fecha'])
    
    dfs.append(df3)
    print(mes)
    
# Concatenate the list of DataFrames into a single DataFrame
result_df = pd.concat(dfs, ignore_index=True)

0
1
2
3
4
5
6
7
8
9
10
11
12


In [14]:
result_df

,fecha,region_code,product_id,product_name,dtd_cost,benchmark,gross_price,net_price,quantity,gmv,expected_dtd_cost
0,2024-04-01,SPO,-314593,SPO || Banana Prata Padrão Kg,5.4074,8.3000,7.7790,7.7790,273.0000,2123.67000,NaN
1,2024-04-01,BHZ,281477,BHZ || Berinjela Padrão Kg,3.1118,6.1110,6.0400,6.0400,94.0000,567.76000,NaN
2,2024-04-01,SPO,73947,SPO || Alho Descascado Pacote 1 kg Padrão Unidade,17.2043,22.0000,25.1428,24.9178,987.0000,24816.03000,NaN
3,2024-04-01,VCP,434909,VCP || Alface Americana Unidade Padrão,3.0003,2.7000,3.3808,3.3678,106.0000,358.37000,NaN
4,2024-04-01,SPO,-314656,SPO || Limão Siciliano Padrão Kg,3.4859,6.6700,5.9725,5.9725,240.0000,1433.40000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
102425,2023-04-06,CWB,363247,CWB || Kiwi kg Padrão,9.1005,None,14.7000,14.7000,1.0000,14.70000,NaN
102426,2023-04-13,CWB,363273,CWB || Maracujá Azedo kg Padrão,7.3545,10.0000,10.5000,10.5000,2.0000,21.00000,NaN
102427,2023-04-21,CWB,363207,CWB || Coco Seco kg Padrão,4.0106,None,6.2000,6.2000,2.0000,12.40000,NaN
102428,2023-04-27,VCP,434899,VCP || Abóbora Moranga Kg Padrão,2.5236,None,2.1100,2.1100,15.0000,31.65000,NaN


In [15]:
result_df[['dtd_cost','benchmark', 'gross_price', 'net_price', 'quantity', 'gmv','expected_dtd_cost']] = result_df[['dtd_cost','benchmark', 'gross_price', 'net_price', 'quantity', 'gmv','expected_dtd_cost']].astype(float)

In [16]:
# Replace NaN values in 'expected_dtd_cost' with corresponding 'dtd_cost' values
result_df['expected_dtd_cost'].fillna(result_df['dtd_cost'], inplace=True)

# Create 'bench_quantity' column based on condition
result_df['bench_quantity'] = result_df.apply(lambda row: row['quantity'] if not pd.isnull(row['benchmark']) else 0, axis=1)

# COGS
result_df['cogs'] = result_df['dtd_cost'] * result_df['quantity']
result_df['cogs_expected'] = result_df['expected_dtd_cost'] * result_df['quantity']

# Convert 'fecha' to datetime
result_df['fecha'] = pd.to_datetime(result_df['fecha'])

# Extract month and year from 'fecha' and create a new column with the first day of the month
result_df['month_year'] = result_df['fecha'].dt.to_period('M').dt.to_timestamp()

In [17]:
# Group by month and product_id, then calculate weighted averages
df_final = result_df.groupby(['region_code','month_year', 'product_id','product_name']).apply(
    lambda x: pd.Series({
        'dtd_cost': (x['dtd_cost'] * x['gmv']).sum() / x['gmv'].sum(),
        'benchmark': (x['benchmark'] * x['gmv']).sum() / x['gmv'].sum(),
        'gross_price': (x['gross_price'] * x['gmv']).sum() / x['gmv'].sum(),
        'net_price': (x['net_price'] * x['gmv']).sum() / x['gmv'].sum(),
        'expected_dtd_cost': (x['expected_dtd_cost'] * x['gmv']).sum() / x['gmv'].sum(),
        'quantity': x['quantity'].sum(),
        'bench_quantity': x['bench_quantity'].sum(),
        'gmv': x['gmv'].sum(),
        'cogs': x['cogs'].sum(),
        'cogs_expected': x['cogs_expected'].sum()
    })
).reset_index()

In [18]:
df_final.month_year.unique()

array(['2023-04-01T00:00:00.000000000', '2023-05-01T00:00:00.000000000',
       '2023-06-01T00:00:00.000000000', '2023-07-01T00:00:00.000000000',
       '2023-08-01T00:00:00.000000000', '2023-09-01T00:00:00.000000000',
       '2023-10-01T00:00:00.000000000', '2023-11-01T00:00:00.000000000',
       '2023-12-01T00:00:00.000000000', '2024-01-01T00:00:00.000000000',
       '2024-02-01T00:00:00.000000000', '2024-03-01T00:00:00.000000000',
       '2024-04-01T00:00:00.000000000'], dtype='datetime64[ns]')

In [19]:
df_final[['month_year','region_code', 'product_id', 'product_name','dtd_cost','expected_dtd_cost',
          'benchmark', 'gross_price', 'net_price','quantity','gmv',
          'bench_quantity', 'cogs', 'cogs_expected']].to_excel('info_contribution.xlsx',index=False)